# HALITE 4 DATASET BUILDER

In [ ]:
import json
import numpy as np
from types import SimpleNamespace  
import random
from kaggle_environments.envs.halite.helpers import *
from numpy import asarray
from numpy import save as npsave
import glob
import os


In [ ]:
ROOT_DIR = '/home/rob/data/working/halite/'
MATCH_DIR = ROOT_DIR + '/matches2/'
TRAIN_DATA_DIR = ROOT_DIR + '/train/data/'
TRAIN_LABELS_DIR = ROOT_DIR + '/train/labels/'

teamID = 5234835 # we will imitate this team. the db of epsiode.json's is score > nnnn.

In [ ]:
matches = []
for name in glob.glob(MATCH_DIR + '/*info.json'): 
    matches.append(os.path.basename(name)[:7]) 


# Input/Output functions

In [ ]:
import scipy.ndimage

# player indexes
HALITE = 0
BASES = 1
SHIPS = 2

# ship indexes
POSITION = 0
CARGO = 1

# action coding
action_map = {'': 0, 'NORTH': 1, 'EAST': 2, 'SOUTH': 3, 'WEST': 4, 'CONVERT': 5, 'SPAWN': 6}
inverse_action_map = dict([(v, k) for (k, v) in action_map.items() ])

offsets = [(0,1),(0,-1),(1,0),(-1,0)] + [(1,1), (1,-1), (-1,1), (-1,-1), (2,0), (0,2), (-2,0),(0,-2)]

HALITE_DIV = 500
CARGO_DIV = 500
PH_DIV = 5000

# position decoding
def xy(n):
    return n % 21, n // 21

# mod
def c(n):
    return n % 21

ship_filter=np.array([[0,1,0],[1,1,1],[0,1,0]])

# manhattan distance map on flat torus geometry
# https://stackoverflow.com/a/62524083
def mdc(x, y, size_x, size_y):
    a, b = divmod(size_x, 2)
    x_template = np.r_[:a+b, a:0:-1] # [0 1 2 1] for size_x == 4 and [0 1 2 2 1] for size_x == 5
    x_template = np.roll(x_template, x) # for x == 2, size_x == 8: [2 1 0 1 2 3 4 3]
    a, b = divmod(size_y, 2)
    y_template = np.r_[:a+b, a:0:-1]
    y_template = np.roll(y_template, y)
    return np.add.outer(x_template, y_template)

def dm_from_sys(arr_sy):
    # return normalised manhattan distance to nearest shipyard from an array of shipyards
    dms = []
    r, c = np.nonzero(arr_sy)
    for i in range(len(r)):
        dms.append(mdc(r[i],c[i],21,21))
    if len(dms)>0: 
        return np.amin(dms,axis=0)
    else:
        return np.ones((21,21), dtype = np.float32)

def getHalite(obs):
    return (np.fromiter(obs['halite'], dtype = np.float32).reshape(21, 21).T / HALITE_DIV).astype(np.half)

def getAllObjects3(obs, conf, config):
    ships = {};                                                # List of xy of all ships (not bases)
    ship = np.zeros((conf.size, conf.size), dtype=np.int8)     # MxM of ships for player P, at ship locations
    base = np.zeros((conf.size, conf.size), dtype=np.int8 )    # MxM of bases for player P, at base locations
    cargo = np.zeros((conf.size, conf.size), dtype=np.half)    # MxM of cargo per ship, at ship locations
    ph = np.zeros((conf.size, conf.size), dtype=np.half)       # MxM of overall halite per player, at ship locations
    threat = np.zeros((conf.size, conf.size), dtype=np.half)
    board = Board(obs, config)
    
    for pidx, p in enumerate(obs['players']):                  # for each player (0-3)
        for b_idx, b in enumerate(p[BASES].values()):          #   for each base
            x, y = xy(b)                                       #     set base and ph       
            base[x, y] = pidx + 1                              #   for each ship
            ph[x, y] = p[HALITE] / PH_DIV                      #     set ship, cargo, ph, ships
                                                              # 
        for k, v in p[SHIPS].items():
            x, y = xy(v[POSITION])
            ship[x, y] = pidx + 1
            cargo[x, y] = v[CARGO] / CARGO_DIV 
            ph[x, y] = p[HALITE] / PH_DIV
            ships[k] = v[POSITION]

            # Threat count
            sh = board.ships[k]
            count_threat = 0
            for j, oset in enumerate(offsets):
                nei = sh.cell.neighbor(oset).ship
                if nei is None: continue
                if nei.player_id == sh.player_id: continue
                if nei.halite >= sh.halite: continue
                if j<=3: count_threat += 0.1
                else: count_threat += 0.07

            threat[x, y] = count_threat 
            
    return ships, ship, base, cargo, ph, threat

def processStep3(obs, conf, config):   
    step = obs['step']
    halite = getHalite(obs)
    ships, ship, base, cargo, ph, threat  = getAllObjects3(obs, conf, config)

    return (step, halite, ship, base, cargo, ph, threat)

def convolve_norm(mat, filt=ship_filter, mult=1):
    for _ in range(mult):
        mat += scipy.ndimage.convolve(mat, filt, mode='wrap',cval=0.0)
    return (mat-mat.min())/(mat.max()-mat.min())

SHIP_START = 4
BASE_START = 8
PLAYER_CARGO_START = 12
 
N_INPUTS = 29

N_POLICY_CHOICES = len(action_map)

dms = {}    

def getInputStack3(obs, match, step, halite, ship, base, cargo, ph, threat, action_map, 
                      first_player):

    first_player += 1 # from 0-3 to 1-4
    other_players = [p for p in [1, 2, 3, 4] if p != first_player] # shuffle other players (redundant)
    random.shuffle(other_players)   

    input_stack = np.zeros((N_INPUTS, 21, 21), dtype = np.float32)
    input_stack[0] = halite
    
    input_stack[1] = (ship==first_player) * halite
    input_stack[2] = (ship==first_player) * cargo
    input_stack[3] = (ship == first_player)  
    input_stack[4] = (base == first_player)  
    input_stack[5] = (ship==first_player) * ph
    input_stack[6] = np.logical_and(input_stack[3],(cargo==0))

    for i in range(3): 
        input_stack[7] += ((ship==other_players[i]))
        input_stack[8+i] = ((ship==other_players[i])) 

    for i in range(3): 
        input_stack[11] += ((ship==other_players[i]) * halite)
        input_stack[12] += ((ship==other_players[i]) * cargo)
        input_stack[13] += (base == other_players[i])
        input_stack[14] += ((ship==other_players[i]) * ph)
        input_stack[15] += np.logical_and(input_stack[8+i],(cargo==0))
        
    input_stack[16] = np.min([step/387,1])

    # 10 - manhattan distance from our closest base 
    input_stack[17] = dm_from_sys(input_stack[4,:,:])
    # 17 - manhattan distance from their closest base 
    input_stack[18] = dm_from_sys(input_stack[13,:,:])

    input_stack[19] = 1*(scipy.ndimage.convolve(input_stack[3,:,:].copy(), ship_filter, mode='wrap',cval=0.0) - \
                            input_stack[7])>0
    input_stack[20] = 1*(scipy.ndimage.convolve(input_stack[7,:,:].copy(), ship_filter, mode='wrap',cval=0.0) - \
                            input_stack[3])>0
    
    # ours and their dominance 
    if np.sum(input_stack[3,:,:]) > 0:
        input_stack[21] = convolve_norm(input_stack[3,:,:].copy(), mult=3)
    if np.sum(input_stack[7,:,:]) > 0:
        input_stack[22] = convolve_norm(input_stack[7,:,:].copy(), mult=3)

    # last action 
    input_stack[23:28] = getPreviousActions(obs, match, step, first_player-1)
    
    # Number of lighter enemy ships within manhattan<=2
    input_stack[28] = threat
        
    return input_stack 

def getPreviousActions(obs, match, step, first_player):
    action_ship = np.zeros((21,21), dtype = np.int8)
    # ship actions
    for player in [first_player]: # range(4):
        pa = match['steps'][max(step, 0)][player]['action'] 
        if pa is None: 
            continue
        for k, v in obs['players'][player][SHIPS].items():
            if k in pa.keys():
                if pa[k] in action_map.keys():
                    x, y = xy(v[POSITION])
                    action_ship[x, y] = action_map[pa[k]]
                else:
                    print("unknown ship action {}".format(pa[k]))
                
    output_stack = np.zeros((5, 21, 21), dtype = np.float32)
    for i in range(1,5): # NESW
        output_stack[i,:,:] = 1*action_ship==i

    output_stack[0,:,:] = 1*action_ship==-1 # STILL
        
    return output_stack

def getActions3(obs, conf, match, step, first_player):    
    action_ship = np.zeros((conf.size, conf.size), dtype = np.int8)
    action_base = np.zeros((conf.size, conf.size), dtype = np.int8)
    
    ship_action_dict = {}
    
    # ship actions
    for player in [first_player]: #range(0, 4):
        pa = match['steps'][step + 1][player]['action']
        if pa is None:
            continue;
        for k, v in obs['players'][player][SHIPS].items():
            if k in pa.keys():
                if pa[k] in action_map.keys():
                    x, y = xy(v[POSITION])
                    action_ship[x, y] = action_map[pa[k]]
                    ship_action_dict[k] = action_map[pa[k]]
                else:
                    print("unknown ship action {}".format(pa[k]))
            else:
                x, y = xy(v[POSITION])
                action_ship[x, y] = -1
                ship_action_dict[k] = -1

        # base actions
        for k, v in obs['players'][player][BASES].items():
            if k in pa.keys():
                if pa[k] in action_map.keys():
                    if pa[k] == 'SPAWN':
                        x, y = xy(v)
                        action_base[x, y] = 1
                    else:
                        print('unknown ship action')
                else:
                    print("unknown base action {}".format(pa[k]))
    
    return action_ship, action_base, ship_action_dict

def getOutputStack3(obs, conf, match, step, first_player):

    output_stack = np.zeros((N_POLICY_CHOICES, 21, 21), dtype = np.float32)

    action_ship, action_base, ship_action_dict = getActions3(obs, conf, match, step, first_player)
    
    for i in range(1,N_POLICY_CHOICES-1): # NESW
        output_stack[i,:,:] = 1*action_ship==i
    
    output_stack[0,:,:] = 1*action_ship==-1 # STILL

    output_stack[N_POLICY_CHOICES-1,:,:] = 1*action_base==1 # SPAWN

    return output_stack[:5,:,:]

def player_index_from_matchid(match_info, team_id):
    for p, agents in enumerate(match_info["agents"]):
        submission = agents["submission"]
        if submission["teamId"] == team_id:
            return p
    return -1
    



## Dataset creation functions

In [ ]:
import collections

def matchReadJSON(fn):
    
    with open(fn, "r") as read_file:
        match_info = json.load(read_file)

    return match_info

def process_match_1p(match_id):
    save=True
    match_id = str(match_id)
    match = matchReadJSON(MATCH_DIR + match_id +'.json')
    match_info = matchReadJSON(MATCH_DIR + match_id +'_info.json')
    config = match["configuration"]
    conf = SimpleNamespace(**config)   

    playerID = player_index_from_matchid(match_info, teamID)

    for s in range(len(match["steps"])-1):
        obs = match["steps"][s][0]["observation"]

        # convert to game format
        step, halite, ship, base, cargo, ph, threat = processStep3(obs, conf, config)

        # featurise
        input_stack = getInputStack3(obs, match, step, halite, ship, base, cargo, ph, threat, action_map,
                                   first_player = playerID)

        output_stack = getOutputStack3(obs, conf, match, step, first_player = playerID)

        if save:
            # save to npy file
            npsave(TRAIN_DATA_DIR + str(match_id) + '_p' + str(playerID) + '_' + str(s).zfill(3) + '.npy', input_stack)
            npsave(TRAIN_LABELS_DIR + str(match_id)  + '_p' + str(playerID) + '_' + str(s).zfill(3) + '.npy', output_stack)



In [ ]:
import multiprocessing

pool = multiprocessing.Pool()

result = pool.map(process_match_1p, matches) # imitate a target team in these games
